In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [2]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate_all.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'NewSolver', 'QTOSimplifySolver', 'QTOSimplifyDiscardSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

best_solution_probs                            \
                                     HeaSolver PenaltySolver ChocoSolver   
pkid variables constraints                                                 
0    6         3                      4.257812      8.466797   93.867188   
1    14        9                      0.039062      0.283203   59.257812   
2    21        12                     0.000000      0.000000   25.263672   

                                                         \
                            NewSolver QTOSimplifySolver   
pkid variables constraints                                
0    6         3            99.785156         99.150391   
1    14        9            92.998047         85.253906   
2    21        12           57.021484               NaN   

                                                    in_constraints_probs  \
                           QTOSimplifyDiscardSolver            HeaSolver   
pkid variables constraints                                                 
0    6         3                          99.638672            21.152344   
1    14        9                          59.990234             0.156250   
2    21        12                               NaN             0.000000   

                                                                \
                           PenaltySolver ChocoSolver NewSolver   
pkid variables constraints                                       
0    6         3               22.128906       100.0     100.0   
1    14        9                1.708984       100.0     100.0   
2    21        12               0.019531       100.0     100.0   

                                                                       \
                           QTOSimplifySolver QTOSimplifyDiscardSolver   
pkid variables constraints                                              
0    6         3                       100.0                    100.0   
1    14        9                       100.0                    100.0   
2    21        12                        NaN                      NaN   

                                   ARG                                      \
                             HeaSolver PenaltySolver ChocoSolver NewSolver   
pkid variables constraints                                                   
0    6         3             47.842510     51.076620    0.071964  0.002880   
1    14        9             91.655058     61.277429    0.123565  0.014760   
2    21        12           164.076592    145.602018    0.352140  0.125671   

                                                                       \
                           QTOSimplifySolver QTOSimplifyDiscardSolver   
pkid variables constraints                                              
0    6         3                    0.015739                 0.004151   
1    14        9                    0.027335                 0.258442   
2    21        12                        NaN                      NaN   

                           iteration_count                            \
                                 HeaSolver PenaltySolver ChocoSolver   
pkid variables constraints                                             
0    6         3                     301.0         120.9       112.1   
1    14        9                     301.0         107.7       105.5   
2    21        12                    201.0         104.1       115.3   

                                                        \
                           NewSolver QTOSimplifySolver   
pkid variables constraints                               
0    6         3               142.4              89.6   
1    14        9               245.1             267.6   
2    21        12              201.0               NaN   

                                                     
                           QTOSimplifyDiscardSolver  
pkid variables constraints                           
0    6         3                               39.9  
1    14        9

In [3]:

# 读入 CSV 文件
df = pd.read_csv('evaluate_TSP.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

FileNotFoundError: [Errno 2] No such file or directory: 'evaluate_TSP.csv'

In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eva_SCP.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

best_solution_probs                         \
                                 PenaltySolver CyclicSolver HeaSolver   
pkid variables constraints                                              
0    8         4                      0.266113     7.778320  1.613770   
1    15        8                      0.002713     0.751411  0.024414   
2    20        12                     0.000000     0.015024  0.002504   

                                       in_constraints_probs               \
                           ChocoSolver        PenaltySolver CyclicSolver   
pkid variables constraints                                                 
0    8         4             48.641276             7.924805    14.165039   
1    15        8             17.677774             1.459418     2.425130   
2    20        12            21.620302             0.826322     0.923978   

                                                           ARG               \
                           HeaSolver ChocoSolver PenaltySolver CyclicSolver   
pkid variables constraints                                                    
0    8         4            6.608887   99.992188   3001.552713  2355.818696   
1    15        8            0.295681   99.993357    652.089984   549.923065   
2    20        12           0.025040   99.989443    190.214544   167.258469   

                                                    iteration_count  \
                              HeaSolver ChocoSolver   PenaltySolver   
pkid variables constraints                                            
0    8         4            3923.325473    0.593665      130.925000   
1    15        8            1072.115783    0.398720      130.972222   
2    20        12            372.264815    0.234680      135.615385   

                                                               
                           CyclicSolver HeaSolver ChocoSolver  
pkid variables constraints                                     
0    8         4             127.525000     201.0   24.213333  
1    15        8             130.111111     201.0   25.557823  
2    20        12            134.897436     201.0   25.689189

In [ ]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

0.27 & 7.78 & 1.61 & 48.64 & 7.92 & 14.17 & 6.61 & 99.99 & 3002 & 2356 & 3923 & 0.59 & 131 & 128 & 201 & 24.21
0.00 & 0.75 & 0.02 & 17.68 & 1.46 & 2.43 & 0.30 & 99.99 & 652 & 550 & 1072 & 0.40 & 131 & 130 & 201 & 25.56
0.00 & 0.02 & 0.00 & 21.62 & 0.83 & 0.92 & 0.03 & 99.99 & 190 & 167 & 372 & 0.23 & 136 & 135 & 201 & 25.69


In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eva_old.csv')

df = df.drop(columns=['pbid'])

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean'
})
grouped_df
# 假设grouped_df是一个Pandas DataFrame
grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
grouped_df = grouped_df.drop(columns=['layers'])
## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




method_order = ['penalty', 'cyclic', 'HEA', 'commute']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

best_solution_probs                       \
                                       penalty     cyclic       HEA   
pkid variables constraints                                            
0    6         3                      3.789062  13.828125  8.906250   
1    15        8                      0.136719   0.097656  0.000000   
2    21        12                     0.000000   0.019531  0.000000   
3    28        15                     0.000000   0.019531  0.000000   
4    12        6                      2.441406  10.058594  0.260417   
5    15        9                      1.757812   0.162760  0.032552   
6    24        12                     0.000000   0.058594  0.000000   
7    28        16                     0.019531   0.000000  0.000000   
8    8         6                      1.660156   5.986328  1.035156   
9    18        9                      0.009766   9.414062  0.000000   
10   24        11                     0.009766   0.000000  0.000000   
11   27        12                     0.000000   0.000000  0.000000   

                                      in_constraints_probs             \
                              commute              penalty     cyclic   
pkid variables constraints                                              
0    6         3            60.058594            22.021484  58.857422   
1    15        8            46.015625             0.468750   1.816406   
2    21        12           10.390625             0.039062   0.341797   
3    28        15           12.158203             0.000000   0.566406   
4    12        6            88.281250             4.166667  39.973958   
5    15        9            50.488281             2.766927   2.311198   
6    24        12           13.876953             0.009766  20.292969   
7    28        16            1.259766             0.029297  37.109375   
8    8         6            56.308594             5.175781  18.037109   
9    18        9            31.201172             0.048828  39.326172   
10   24        11            3.642578             0.009766  59.482422   
11   27        12            1.816406             0.000000  49.531250   

                                                       ARG              \
                                  HEA commute      penalty      cyclic   
pkid variables constraints                                               
0    6         3            26.435547   100.0    38.961044   15.258196   
1    15        8             0.097656   100.0   107.684447   70.948516   
2    21        12            0.000000   100.0   144.943700   94.616833   
3    28        15            0.000000   100.0   176.658959   97.666260   
4    12        6             0.358073   100.0   697.809245  216.966471   
5    15        9             0.032552   100.0   746.790202  620.558594   
6    24        12            0.000000   100.0  2591.484766  478.821094   
7    28        16            0.000000   100.0  2365.513525  500.637939   
8    8         6             4.462891   100.0    53.783871   32.273939   
9    18        9             0.039062   100.0   117.935604   37.785087   
10   24        11            0.000000   100.0   162.481877   24.699744   
11   27        12            0.000000   100.0   162.490013   30.530602   

                                                  iteration_count              \
                                    HEA   commute         penalty      cyclic   
pkid variables constraints                                                      
0    6         3              44.912728  0.164989      143.600000  150.800000   
1    15        8             149.811792  0.299001      143.300000  140.200000   
2    21        12            180.885226  0.501108      137.000000  139.500000   
3    28        15            257.122339  0.434504      142.100000  141.800000   
4    12        6            1144.250000  0.058594      147.666667  148.666667   
5    15        9            1727.261556  0.247559      148.333333  143.666667   
6    24        12        

In [ ]:
pivot_df['ARG']

,,,penalty,cyclic,HEA,commute
pkid,variables,constraints,,,,
0,6,3,38.961044,15.258196,44.912728,0.164989
1,15,8,107.684447,70.948516,149.811792,0.299001
2,21,12,144.943700,94.616833,180.885226,0.501108
3,28,15,176.658959,97.666260,257.122339,0.434504
4,12,6,697.809245,216.966471,1144.250000,0.058594
5,15,9,746.790202,620.558594,1727.261556,0.247559
6,24,12,2591.484766,478.821094,3090.787354,0.469678
7,28,16,2365.513525,500.637939,3834.283008,0.559863
8,8,6,53.783871,32.273939,59.376893,0.136039


In [ ]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('eval_eliminate copy 2.csv')

df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

grouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
    "successrate": 'mean',
})
grouped_df
# 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




method_order = ['penalty', 'cyclic', 'HEA', 'commute']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

pivot_df

KeyError: 'layers'